In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain_core.messages import HumanMessage, ToolMessage, AIMessage # Import AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Literal, List, Dict, Any, Optional
import operator
import json
import re
from datetime import datetime # Import datetime
from typing_extensions import TypedDict
from typing import Annotated, List, Sequence, Tuple, TypedDict, Union
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from dotenv import load_dotenv, find_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.tools import tool
from langchain.agents import create_openai_functions_agent
from langchain.tools.render import format_tool_to_openai_function
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages.base import BaseMessage
from langchain_core.messages.function import FunctionMessage
from langchain_core.messages import HumanMessage, SystemMessage
import operator
import functools
from langchain_community.tools import DuckDuckGoSearchRun
import uuid



In [8]:


search = DuckDuckGoSearchRun()
res = search.invoke("Full stack roadmap")
print(type(res), res)

<class 'str'> Learn how to become a full stack web developer in 2025 with this comprehensive guide. It covers front-end, back-end, database, and version control systems, as well as popular technologies like MERN, MEAN, HTML, CSS, JavaScript, and more. This Java Full Stack Developer roadmap has been meticulously crafted through extensive research into emerging technologies and industry trends. It anticipates the future demands of the tech landscape, ensuring that you're equipped with the skills and knowledge needed to stay ahead of the curve and thrive in the rapidly evolving field of ... Here's your step-by-step roadmap to becoming a confident, job-ready full stack developer. What Does "Full Stack" Mean? The "stack" refers to the collection of technologies used to build a complete web application. It typically includes: Frontend (what users see): The layout, buttons, text, visuals, and animations — everything on screen. Everything About Full Stack Developer Roadmap in 2025. The full st

In [23]:
def webSearchTool(query:str):
    """
    Perform a web search using DuckDuckGo.

    Args:
        query (str): The search query string.

    Returns:
        str: The search results returned by DuckDuckGo.
    """
    search = DuckDuckGoSearchRun()
    res = search.invoke(query)
    return res

In [48]:
# --- 3. Define the prompt for tool calling ---
tool_calling_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful AI assistant, who is designed to create planners. The user give anything for which you need to make a detailed plan for the user. For example, the user can ask "I want to learn DSA to crack interviews." or "I want to visit Ooty, make a itenary".

    You have access to the following tools:
    {tools}

    When the user asks a question that requires a tool, respond by setting the "tool_calling_required" flag as true.
    For example:
    <tool_code>
    {{
        "tool_calling_required" : true,
        "tool_name": "webSearchTool",
        "args": {{
            "query": "What is the roadmap for learning Full Stack development"
        }}
    }}
    </tool_code>
    Do NOT make up tool calls if you don't have enough information.
    If you have just received tool outputs, synthesize a natural language response for the user based on the chat history and the tool outputs.
    NOTE: YOUR FINAL ANSWER SHOULD BE A JSON OBJECT WHICH I CAN DIRECTLY PARSE WITH json.loads PYTHON METHOD. YOUR FINAL ANSWER WILL BE A DIRECT INPUT TO A FRONTEND APPLICAION WHICH WILL BE MAPPED TO UI ELEMENTS. IF YOU HAVE THE FINAL ANSWER, "tool_calling_required" FIELD SHOULD BE false.
    The Structure of the final ans:
    {{
        "tool_calling_required" : false,
        "final_answer": <List of Tasks Object>
    }}
    JSON Structure of Task Object:
    {{
        "title" : <Title of Task>
        "sub_tasks" : <List of Actions to complete this Task>
    }} 
     
     Example of a final answer:
     {{
        "tool_calling_required" : false,
        "final_answer": [
            {{
                "title": "Study Time complexity",
                "sub_tasks": [
                    "Study big O notation",
                    "Study about different types of timeplexity like O(N), O(logN) etc"
                ] 
            }},
            {{
                "title": "Arrays",
                "sub_tasks": [
                    "Searching in array",
                    "Sorting an array",
                    "Prefix sum",
                ] 
            }},...
        ]
     }}
    """),
    ("placeholder", "{chat_history}"), # Placeholder for full chat history
    ("user", "{input}") # Only original user input, subsequent turns are in chat_history
])

In [49]:
# Format the tool description for the prompt
tools_description = """
- Name: webSearchTool
  Description: Perform a web search using DuckDuckGo.
  Parameters:
    - query: string (REQUIRED) - The query to search on the web.
"""

# --- 1. Initialize Ollama chat model ---
llm = ChatOllama(model="llama3.1", base_url="http://localhost:11434", temperature=0.1)

# --- LangGraph Agent Setup ---

class AgentState(TypedDict):
    input: str # Original user query
    chat_history: Annotated[List[Any], operator.add] # List of messages (Human, AI, Tool)
    tool_calls: List[Dict[str, Any]] # Tool calls detected by LLM
    tool_outputs: List[Any] # Outputs from tool execution
    final_answer: Optional[str] # Final natural language answer

def call_llm(state: AgentState):
    current_input_messages = []
    # If this is the start of the conversation, use the 'input'
    if not state["chat_history"]:
        current_input_messages.append(HumanMessage(content=state["input"]))
    else:
        # If it's a follow-up after a tool call, the relevant info is in chat_history
        # The prompt handles how to interpret the history.
        pass # chat_history is already passed by placeholder

    # Construct messages for the LLM, including history
    messages_for_llm = state["chat_history"] + current_input_messages

    prompt_formatted = tool_calling_prompt.format(
        tools=tools_description,
        input=state["input"], # The original input for initial prompt context
        chat_history=messages_for_llm # Pass the full list of messages
    )
    # Using `invoke` with `input` and `chat_history` args of prompt directly
    # This might need adjustment depending on how `ChatOllama` handles prompt templates
    # For now, let's use the formatted string.
    response = llm.invoke(prompt_formatted)
    content = response.content
    print("Response: ", content[content.find('{'): content.rfind('}') + 1])
    response_json = json.loads(content[content.find('{'): content.rfind('}') + 1])
    print("Success")
    tool_calls = []
    final_answer = None # Start as None

    tool_calling_required = response_json['tool_calling_required']
    print("Tool Calling Required: ", tool_calling_required)
    
    # Simple regex to extract the tool_code block

    if tool_calling_required:
        try:
            tool_calls.append(response_json)
            # Remove the tool_code block from the content
            content_without_tool_code = re.sub(r"<tool_code>.*?</tool_code>", "", content, flags=re.DOTALL).strip()
            # If there's still meaningful content, maybe it's a preamble
            if content_without_tool_code:
                print(f"LLM also said: {content_without_tool_code}") # For debugging
                # Decide if this content should be part of chat history or discarded
                # For now, we prioritize tool_calls
        except json.JSONDecodeError:
            print("Warning: Could not parse tool_code JSON. Treating as regular text.")
            final_answer = content # Treat as final answer if tool call parsing fails
    else:
        final_answer = response_json['final_answer'] # No tool call detected, assume it's a direct answer

    # Append AI's full response (including potential tool_code) to chat history
    new_chat_history_entry = [AIMessage(content=content)]

    return {
        "chat_history": new_chat_history_entry, # Only add the *latest* LLM message
        "tool_calls": tool_calls,
        "final_answer": final_answer
    }

def call_tool(state: AgentState):
    tool_outputs = []
    # Loop through tool calls (though often there's just one per turn)
    for tool_call in state["tool_calls"]:
        tool_name = tool_call.get("tool_name")
        args = tool_call.get("args", {})

        if tool_name == "webSearchTool":
            output = webSearchTool(**args)
            tool_outputs.append(output)
            tool_message_content = {"tool_calls": [tool_call], "output": output}
        else:
            error_msg = f"Error: Tool '{tool_name}' not found."
            tool_outputs.append(error_msg)
            tool_message_content = {"tool_calls": [tool_call], "error": error_msg}

    # Crucial: Return a ToolMessage with the output to be added to chat history
    # This is how the LLM "sees" the result of its tool call.
    return {
        "tool_outputs": tool_outputs,
        "chat_history": [ToolMessage(tool_message_content, tool_call_id=str(uuid.uuid4()))] # Unique ID
    }

# def jsonParserAgent(state: AgentState):

def decide_next_step(state: AgentState):
    if state["tool_calls"]:
        print("Decision: Calling Tool")
        return "call_tool"
    elif state["final_answer"]:
        print("Decision: Ending Chat (Final Answer)")
        return "end_chat"
    else:
        # Fallback for unexpected scenarios, or if LLM didn't give a final answer
        # but also didn't call a tool. Could mean it needs more turns.
        # For a roadmap, this might be a loop for refinement.
        print("Decision: No tool call, no final answer. Trying LLM again or ending.")
        # If you want to loop back to LLM to try to generate a final answer
        # based on history, return "call_llm"
        # If you want to stop if nothing useful happens, return END
        return END # Changed to END for safety in this example.

In [50]:
# Build the LangGraph
workflow = StateGraph(AgentState)

workflow.add_node("call_llm", call_llm)
workflow.add_node("call_tool", call_tool)

workflow.set_entry_point("call_llm")

workflow.add_conditional_edges(
    "call_llm",
    decide_next_step, # Function to decide next edge
    {
        "call_tool": "call_tool",
        "end_chat": END
    }
)

# After tool execution, always go back to LLM for synthesis
workflow.add_edge("call_tool", "call_llm")

app = workflow.compile()

In [51]:

# --- Example Usage ---
import uuid # For generating tool_call_id
from datetime import datetime

# Test 1: Simple query, no tool needed
print("--- Test 1: Direct Answer ---")
inputs = {"input": "I want to learn full stack development", "chat_history": []}
# for s in app.stream(inputs):
#     print(s)
# print("\n")

result = app.invoke(inputs)

--- Test 1: Direct Answer ---
Response:  {
  "tool_calling_required" : true,
  "tool_name": "webSearchTool",
  "args": {
    "query": "Full Stack Development Roadmap"
  }
}
Success
Tool Calling Required:  True
LLM also said: {
  "tool_calling_required" : true,
  "tool_name": "webSearchTool",
  "args": {
    "query": "Full Stack Development Roadmap"
  }
}
Decision: Calling Tool
Response:  {
  "tool_calling_required" : false,
  "final_answer": [
    {
      "title": "Understand Full Stack Development",
      "sub_tasks": [
        "Read the definition of Full Stack Development from the guide provided by the tool output.",
        "Understand the components of a full stack, including frontend and backend."
      ]
    },
    {
      "title": "Frontend Skills",
      "sub_tasks": [
        "Learn HTML, CSS, and JavaScript as per the guide's recommendations.",
        "Explore popular front-end frameworks like React or Angular."
      ]
    },
    {
      "title": "Backend Skills",
      "s

In [52]:
result['final_answer']

[{'title': 'Understand Full Stack Development',
  'sub_tasks': ['Read the definition of Full Stack Development from the guide provided by the tool output.',
   'Understand the components of a full stack, including frontend and backend.']},
 {'title': 'Frontend Skills',
  'sub_tasks': ["Learn HTML, CSS, and JavaScript as per the guide's recommendations.",
   'Explore popular front-end frameworks like React or Angular.']},
 {'title': 'Backend Skills',
  'sub_tasks': ['Study backend languages such as Node.js, Python, or Ruby as mentioned in the guide.',
   'Learn about databases and version control systems like Git.']},
 {'title': 'Practice with Projects',
  'sub_tasks': ['Build projects that integrate frontend and backend skills, as suggested by the guide.',
   'Participate in coding challenges or contribute to open-source projects to gain experience.']}]